# **CASE: CANCELAMENTO EM TELECOM**

---

### Instalação de bibliotecas

In [ ]:
# Para cálculo do VIF
install.packages("rms")
# Para cálculo do KS e AUC
install.packages("ROCR")

### Carregamento de bibliotecas

In [ ]:
library(rms)
library(ROCR)

### Leitura da base de dados

In [ ]:
dados_telecom <- read.table(file = "Cancelamento_Telecom.txt",
                            sep = "\t",
                            dec = ".",
                            header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_telecom)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_telecom)

In [ ]:
# Tabela de frequências: gênero
table(dados_telecom$GENERO)
prop.table(table(dados_telecom$GENERO))

In [ ]:
# Tabela de frequências: tipo de plano
table(dados_telecom$TIPO_PLANO)
prop.table(table(dados_telecom$TIPO_PLANO))

In [ ]:
# Tabela de frequências: cancelamento
table(dados_telecom$CANCELOU)
prop.table(table(dados_telecom$CANCELOU))

### Análise bivariada: variável resposta vs. variáveis explicativas

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 20, repr.plot.height = 7)

In [ ]:
# Gráficos de boxplots
par(mfrow = c(1,4), cex = 1)
boxplot(SCORE_CREDITO        ~ CANCELOU, data = dados_telecom, col = "darkturquoise", main = "Score de Crédito")
boxplot(IDADE                ~ CANCELOU, data = dados_telecom, col = "darkturquoise", main = "Idade")
boxplot(TEMPO_RELACIONAMENTO ~ CANCELOU, data = dados_telecom, col = "darkturquoise", main = "Tempo de Relacionamento")
boxplot(RENDA                ~ CANCELOU, data = dados_telecom, col = "darkturquoise", main = "Renda")

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

In [ ]:
# Tabela de frequências: gênero versus resposta (somando 100% nas linhas)
table(dados_telecom$GENERO, dados_telecom$CANCELOU)
prop.table(table(dados_telecom$GENERO, dados_telecom$CANCELOU), 1)

In [ ]:
# Tabela de frequências: tipo de plano versus resposta (somando 100% nas linhas)
table(dados_telecom$TIPO_PLANO, dados_telecom$CANCELOU)
prop.table(table(dados_telecom$TIPO_PLANO, dados_telecom$CANCELOU), 1)

### Modelo de regressão logística múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(CANCELOU ~
                     SCORE_CREDITO +
                     GENERO +
                     IDADE +
                     TEMPO_RELACIONAMENTO +
                     TIPO_PLANO +
                     RENDA,
                   family = binomial(link = 'logit'),
                   data = dados_telecom)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando SCORE_CREDITO
regressao_2 <- glm(CANCELOU ~
                     GENERO +
                     IDADE +
                     TEMPO_RELACIONAMENTO +
                     TIPO_PLANO +
                     RENDA,
                   family = binomial(link = 'logit'),
                   data = dados_telecom)

summary(regressao_2)

In [ ]:
# Ajuste do modelo 3: retirando GENERO
regressao_3 <- glm(CANCELOU ~
                     IDADE +
                     TEMPO_RELACIONAMENTO +
                     TIPO_PLANO +
                     RENDA,
                   family = binomial(link = 'logit'),
                   data = dados_telecom)

summary(regressao_3)

In [ ]:
# Ajuste do modelo 4: retirando TEMPO_RELACIONAMENTO
regressao_4 <- glm(CANCELOU ~
                     IDADE +
                     TIPO_PLANO +
                     RENDA,
                   family = binomial(link = 'logit'),
                   data = dados_telecom)

summary(regressao_4)

In [ ]:
# Avaliação de colinearidade no modelo 4
data.frame(VIF = vif(regressao_4))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os parâmetros referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_4$coefficients[beta + 1] - 1.96 * coef(summary(regressao_4))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_4$coefficients[beta + 1] + 1.96 * coef(summary(regressao_4))[beta + 1, "Std. Error"])

### Aplicação do modelo e definição de resposta predita

In [ ]:
# Aplicação do modelo na base (criação de uma nova coluna chamada "probabilidade")
dados_telecom$probabilidade <- predict(regressao_4,
                                       dados_telecom,
                                       type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_telecom$CANCELOU)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_telecom$predito <- as.factor(ifelse(dados_telecom$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho

In [ ]:
# Tabela de classificação
tabela <- table(dados_telecom$CANCELOU, dados_telecom$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_telecom$probabilidade, dados_telecom$CANCELOU)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_telecom$probabilidade, dados_telecom$CANCELOU)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo cliente, que possui plano pré-pago, renda mensal de 5.000 reais e 30 anos de idade.
# Obs.: os nomes das colunas e padrão de conteúdo devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(IDADE      = c(30),
                          TIPO_PLANO = c("Pre_Pago"),
                          RENDA      = c(5000))

In [ ]:
# Aplicação do modelo
novos_dados$PROB_CANCELAR_6M <- predict(regressao_4, novos_dados, type = "response")
View(novos_dados)